We will collect text data using Twitter API.

!pip install tweepy

!pip install openpyxl

In [1]:
import numpy as np
import pandas as pd
import tweepy
import json
from tweepy import OAuthHandler

In [2]:
consumer_key = "ldmkQMs1MbOfZuvcaLhwXmaLI"
consumer_secret = "mgK3PznES1QbKzt1IFu5mQdiJCbcxzdpjNIhIR3PVor17Vqh2r"
access_token = "2893711315-tE2KVQmD5RWIHDujhd7faMEXBUhaLV5iO7iinXC"
access_token_secret = "ioxlcMRZXAm9usVAEHY2ojnUdIo0wmSRpG6Og5M0a9SuC"

In [3]:
auth = tweepy.OAuthHandler( consumer_key , consumer_secret )
auth.set_access_token( access_token , access_token_secret )
api = tweepy.API(auth)

In [4]:
query = "IndVsAus"

In [5]:
Tweets = api.search( query , count=100 , lang='en' , exclude='retweets' , tweet_mode='extended' )
# tweet_mode='extended'
# tweet_mode='compat'

for tweet in Tweets:
    print(tweet,"\n")
    print(".....................................\n")

Status(_api=<tweepy.api.API object at 0x7fbdd07c8a90>, _json={'created_at': 'Fri Dec 25 12:30:38 +0000 2020', 'id': 1342447660867801090, 'id_str': '1342447660867801090', 'full_text': "Steve Smith's Bradmanesque MCG Average, Nathan Lyon Close to 400-Club\n\nAll the numbers ahead of #IndvsAus Boxing Day Test, by @nikhil8483: \n\nhttps://t.co/03bQSvkoGg", 'truncated': False, 'display_text_range': [0, 164], 'entities': {'hashtags': [{'text': 'IndvsAus', 'indices': [96, 105]}], 'symbols': [], 'user_mentions': [{'screen_name': 'nikhil8483', 'name': 'Nikhil Narain', 'id': 601015275, 'id_str': '601015275', 'indices': [126, 137]}], 'urls': [{'url': 'https://t.co/03bQSvkoGg', 'expanded_url': 'https://www.news18.com/cricketnext/news/india-vs-australia-steve-smiths-bradmanesque-mcg-average-nathan-lyon-close-to-400-club-3213686.html', 'display_url': 'news18.com/cricketnext/ne…', 'indices': [141, 164]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://m

Get the tweets and some Attributes

In [6]:
df = pd.DataFrame(columns = ['Tweets' , 'User' , 'User_statuses_count' , 
                            'user_followers' , 'User_location' , 'User_verified' ,
                            'fav_count' , 'rt_count' , 'tweet_date'] )
print(df)

Empty DataFrame
Columns: [Tweets, User, User_statuses_count, user_followers, User_location, User_verified, fav_count, rt_count, tweet_date]
Index: []


In [7]:
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i = i+1
        if i == 1000:
            break
        else:
            pass

In [8]:
stream(data=["IndVsAus"] , file_name='my_tweets')

In [9]:
df.shape

(1000, 9)

In [10]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date
0,RT @its_DRP: Everyone wanted KL Rahul \nAgree ...,Swapnil patil,6000,60,"Kolhapur, India",False,0,3,2020-12-25 12:31:31
1,"Steve Smith's Bradmanesque MCG Average, Nathan...",CricketNext,172551,124942,India,True,1,0,2020-12-25 12:30:38
2,India vs Australia boxing day test preview in ...,Sandanapandi,3299,45,"Madurai South, India",False,1,0,2020-12-25 12:29:37
3,When @stevesmith49 don't allowed to meet her w...,Bond__James Bond,29,2,,False,1,0,2020-12-25 12:28:30
4,RT @SriniMaama16: India XI for Boxing Day Test...,Teetu,8465,9,,False,0,58,2020-12-25 12:28:15


!pip install textblob

Sentiment(polarity, subjectivity)

Polarity score: [-1 , +1]

Subjectivity: [0.0 , 1.0] 
where 0.0 is very Objective and 1.0 is very Subjective.

In [11]:
from textblob import TextBlob

In [12]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [13]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [14]:
df['clean_tweet'] = df['Tweets'].apply(lambda x : clean_tweet(x))

In [15]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet
0,RT @its_DRP: Everyone wanted KL Rahul \nAgree ...,Swapnil patil,6000,60,"Kolhapur, India",False,0,3,2020-12-25 12:31:31,RT DRP Everyone wanted KL Rahul Agree He s One...
1,"Steve Smith's Bradmanesque MCG Average, Nathan...",CricketNext,172551,124942,India,True,1,0,2020-12-25 12:30:38,Steve Smith s Bradmanesque MCG Average Nathan ...
2,India vs Australia boxing day test preview in ...,Sandanapandi,3299,45,"Madurai South, India",False,1,0,2020-12-25 12:29:37,India vs Australia boxing day test preview in ...
3,When @stevesmith49 don't allowed to meet her w...,Bond__James Bond,29,2,,False,1,0,2020-12-25 12:28:30,When don t allowed to meet her wife due to cov...
4,RT @SriniMaama16: India XI for Boxing Day Test...,Teetu,8465,9,,False,0,58,2020-12-25 12:28:15,RT India XI for Boxing Day Test Gill debut May...


In [16]:
df['Sentiment'] = df['clean_tweet'].apply(lambda x : analyze_sentiment(x) )
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
0,RT @its_DRP: Everyone wanted KL Rahul \nAgree ...,Swapnil patil,6000,60,"Kolhapur, India",False,0,3,2020-12-25 12:31:31,RT DRP Everyone wanted KL Rahul Agree He s One...,Positive
1,"Steve Smith's Bradmanesque MCG Average, Nathan...",CricketNext,172551,124942,India,True,1,0,2020-12-25 12:30:38,Steve Smith s Bradmanesque MCG Average Nathan ...,Negative
2,India vs Australia boxing day test preview in ...,Sandanapandi,3299,45,"Madurai South, India",False,1,0,2020-12-25 12:29:37,India vs Australia boxing day test preview in ...,Neutral
3,When @stevesmith49 don't allowed to meet her w...,Bond__James Bond,29,2,,False,1,0,2020-12-25 12:28:30,When don t allowed to meet her wife due to cov...,Negative
4,RT @SriniMaama16: India XI for Boxing Day Test...,Teetu,8465,9,,False,0,58,2020-12-25 12:28:15,RT India XI for Boxing Day Test Gill debut May...,Neutral


In [17]:
n = 1
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 Steve Smith's Bradmanesque MCG Average, Nathan Lyon Close to 400-Club

All the numbers ahead of #IndvsAus Boxing Da… https://t.co/RVCIjVd3dQ

Clean tweet:
 Steve Smith s Bradmanesque MCG Average Nathan Lyon Close to 400 Club All the numbers ahead of IndvsAus Boxing Da

Sentiment of the tweet:
 Negative


In [18]:
n = 2
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 India vs Australia boxing day test preview in Tamil
https://t.co/itdmFJvIrV

#INDvsAUS #BoxingDayTest #INDvAUS #BCCI

Clean tweet:
 India vs Australia boxing day test preview in Tamil INDvsAUS BoxingDayTest INDvAUS BCCI

Sentiment of the tweet:
 Neutral


In [29]:
n = 28
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 @RealShubmanGill to make his Test debut after averaging 68.78 in 23 first class matches.
Only four Indian players h… https://t.co/Bi3oWZZFdg

Clean tweet:
 to make his Test debut after averaging 68 78 in 23 first class matches Only four Indian players h

Sentiment of the tweet:
 Positive


In [20]:
df[df.Sentiment == 'Positive'].shape[0]

363

In [21]:
df[df.Sentiment == 'Neutral'].shape[0]

562

In [22]:
df[df.Sentiment == 'Negative'].shape[0]

75